<a href="https://colab.research.google.com/github/Afarrow77/Machine-Learning-Course/blob/main/Amalia_Vil_Prophet_Predecir_el_valor_de_Silver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [25]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [26]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [27]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = "2023-04-19"
start_date = '2016-01-01'
# Descargar el dataframe
silver_df = yf.download('SI=F',start_date, today).reset_index()

silver_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1827,2023-04-12,$25.17,$25.58,$25.17,$25.40,$25.40,14
1828,2023-04-13,$25.54,$25.95,$25.54,$25.87,$25.87,12
1829,2023-04-14,$25.80,$25.90,$25.34,$25.42,$25.42,28
1830,2023-04-17,$25.05,$25.05,$25.05,$25.05,$25.05,0
1831,2023-04-18,$25.18,$25.30,$25.16,$25.25,$25.25,0


In [28]:
silver_df.head(100) #this DataFrame contains 100 rows x 7 columns

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-01-04,$14.10,$14.10,$13.82,$13.82,$13.82,27
1,2016-01-05,$13.95,$13.95,$13.95,$13.95,$13.95,4
2,2016-01-06,$14.00,$14.06,$13.96,$13.96,$13.96,105
3,2016-01-07,$14.34,$14.34,$14.33,$14.34,$14.34,286
4,2016-01-08,$14.06,$14.06,$13.91,$13.91,$13.91,6
...,...,...,...,...,...,...,...
95,2016-05-19,$16.91,$16.91,$16.42,$16.48,$16.48,80
96,2016-05-20,$16.47,$16.52,$16.47,$16.52,$16.52,2
97,2016-05-23,$16.32,$16.43,$16.32,$16.41,$16.41,23
98,2016-05-24,$16.32,$16.32,$16.24,$16.24,$16.24,10


In [29]:
#df = pd.DataFrame(data=jugadores, index=indices) TIP PARA SELECT COLUMNS

#columnas_seleccionadas = df.iloc[:, [0, 3]]
#columnas_seleccionadas

In [30]:
# Validamos que no hay datos vacíos
# silver_df = pd.DataFrame({'Volume': [0]})
print(silver_df.isna()) #si quiero que me sume y aparezca si hay algún valor true .sum()

       Date   Open   High    Low  Close  Adj Close  Volume
0     False  False  False  False  False      False   False
1     False  False  False  False  False      False   False
2     False  False  False  False  False      False   False
3     False  False  False  False  False      False   False
4     False  False  False  False  False      False   False
...     ...    ...    ...    ...    ...        ...     ...
1827  False  False  False  False  False      False   False
1828  False  False  False  False  False      False   False
1829  False  False  False  False  False      False   False
1830  False  False  False  False  False      False   False
1831  False  False  False  False  False      False   False

[1832 rows x 7 columns]


In [33]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = silver_df[["Date", "Open"]]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = silver_df.rename(columns=new_names)

In [34]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [ ]:
df.tail()

,ds,y
1798,2022-10-12 00:00:00+00:00,"$1,279.73"
1799,2022-10-13 00:00:00+00:00,"$1,294.92"
1800,2022-10-14 00:00:00+00:00,"$1,288.05"
1801,2022-10-15 00:00:00+00:00,"$1,297.31"
1802,2022-10-16 00:00:00+00:00,"$1,275.01"


In [35]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = ""
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [36]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode="multiplicative")

In [38]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [39]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=1095)
future.tail()

,ds
2922,2026-04-13
2923,2026-04-14
2924,2026-04-15
2925,2026-04-16
2926,2026-04-17


In [40]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-04
1,2016-01-05
2,2016-01-06
3,2016-01-07
4,2016-01-08
...,...
2922,2026-04-13
2923,2026-04-14
2924,2026-04-15
2925,2026-04-16


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [41]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [42]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2610,2026-04-13,$17.22,$-152.62,$174.50
2611,2026-04-14,$17.22,$-154.02,$172.94
2612,2026-04-15,$17.19,$-153.28,$174.64
2613,2026-04-16,$17.16,$-153.71,$173.36
2614,2026-04-17,$17.11,$-153.69,$174.29


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [43]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [44]:
plot_components_plotly(m, forecast)